In [60]:
run = "high_v3"

cpu_load = "/media/kakn/storage/prometheus_export/" + run + "/processor_cpu_full.json"
message_folder = "/media/kakn/storage/message_export/" + run + "/egress"

In [61]:
range_start = 0
range_stop = 0

import json, os

with open(cpu_load, "r") as f:
    cpu_load = json.load(f)["data"]["result"][0]["values"]
    cpu_load = list(map(lambda x: (x[0], float(x[1])), cpu_load))


for i in range(len(cpu_load) - 19 ):
    if all(cpu_load[j][1] > 0.5 for j in range(i, i + 20)):
        range_start = cpu_load[i][0]
        break

for i in range(len(cpu_load) - 19):
    if all(cpu_load[j][1] < 0.5 for j in range(i, i + 20)):
        range_stop = cpu_load[i-1][0]
        break

range_start = int(range_start * 1000)
range_stop = int(range_stop * 1000)

In [62]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.config("spark.driver.host", "localhost") \
.config("spark.driver.extraJavaOptions", "-Djava.security.manager=allow") \
.config("spark.executor.extraJavaOptions", "-Djava.security.manager=allow") \
.config("spark.driver.bindAddress","localhost")\
.config("spark.ui.port","4050")\
.getOrCreate()

In [63]:
import pyspark.sql.functions as F

message_frame = spark.read.json(message_folder)
message_frame = message_frame.filter((message_frame.kafkaTime >= range_start) & (message_frame.kafkaTime <= range_stop))
message_frame.select(F.min("kafkaTime"), F.max("kafkaTime"), F.count("kafkaTime"), (F.count("kafkaTime") / (F.max("kafkaTime") - F.min("kafkaTime"))).alias("rate mgs/ms")).show(vertical=True)

-RECORD 0------------------------------
 min(kafkaTime)   | 1736694350625      
 max(kafkaTime)   | 1736696062703      
 count(kafkaTime) | 31473698           
 rate mgs/ms      | 18.383331834180453 

